In [10]:
import json
import numpy as np
import sys
sys.path.insert(0, '../library')

from json_file_preprocessing import *
from text_preprocessing import *
from word2vec import *
from helpers import *
from clustering import *
from metrics import *

In [2]:
r = open('../results/fos/fos_5_100.txt', 'r')
fos = json.load(r)
r.close()
print(len(fos))

21364


## Pretrained model

In [3]:
model = load_glove_100()

In [5]:
X = apply_model_w2v(fos,model,100)

Not found 0 words from 35754 words.


In [6]:
print(model.most_similar('mining'))

[('coal', 0.7360191345214844), ('mines', 0.7158533334732056), ('copper', 0.6758443713188171), ('industrial', 0.6711450815200806), ('ore', 0.6710669994354248), ('mine', 0.6709348559379578), ('mineral', 0.6708515286445618), ('exploration', 0.6595709323883057), ('minerals', 0.639697790145874), ('logging', 0.6393759250640869)]


In [21]:
xmeans = apply_XMeans(X,1,100)
clusters = xmeans.get_clusters()
print(len(clusters))

100


In [22]:
cl = get_labels(clusters)
apply_silhouette(X,cl)

The average silhouette_score is : 0.017756829087519063


## Trained Model

In [23]:
my_model = train_word2vec(fos, 'fos_s50i40', size=50, window=2, min_count=1, workers=8, iter=100)
print(get_perplexity(my_model))

1.581179612518452495063554355E+754397


In [15]:
my_X = apply_model_w2v(fos,my_model,100)

Not found 0 words from 35754 words.


In [25]:
print(my_model.wv.most_similar('mining'))

[('ontology', 0.7213804125785828), ('null', 0.7006464004516602), ('migration', 0.6744667291641235), ('analytics', 0.6488808393478394), ('modification', 0.6461018323898315), ('methodology', 0.6411551237106323), ('anthropology', 0.6397405862808228), ('pronoun', 0.6325876712799072), ('breathe', 0.6316117644309998), ('noisy', 0.6288643479347229)]


In [26]:
xmeans = apply_XMeans(my_X,1,1000)
my_clusters = xmeans.get_clusters()
print(len(my_clusters))

87


In [29]:
my_cl = get_labels(my_clusters)
apply_silhouette(my_X,my_cl)

The average silhouette_score is : 0.0144714157531467


In [30]:
print(my_cl)
    

[84. 33. 72. ... 29. 43. 14.]


In [ ]:
print(dunn(X, cl))

In [20]:
from sklearn.metrics.pairwise import euclidean_distances
def delta_fast(ck, cl, distances):
    values = distances[np.where(ck)][:, np.where(cl)]
    values = values[np.nonzero(values)]

    return np.min(values)
    
def big_delta_fast(ci, distances):
    values = distances[np.where(ci)][:, np.where(ci)]
    #values = values[np.nonzero(values)]
            
    return np.max(values)
    
def dunn(points, labels):
    """  points : np.array   np.array([N, p]) of all points
    labels: np.array   np.array([N]) labels of all points
    """
    distances = euclidean_distances(points)
    ks = np.sort(np.unique(labels))
    
    deltas = np.ones([len(ks), len(ks)])*1000000
    big_deltas = np.zeros([len(ks), 1])
    
    l_range = list(range(0, len(ks)))
    
    for k in l_range:
        for l in (l_range[0:k]+l_range[k+1:]):
            deltas[k, l] = delta_fast((labels == ks[k]), (labels == ks[l]), distances)
        
        big_deltas[k] = big_delta_fast((labels == ks[k]), distances)

    di = np.min(deltas)/np.max(big_deltas)
    return di

#k_means = KMeans(n_clusters=i, random_state=0) 
#    k_means.fit(df) #K-means training 
#print(dunn(X, cl))
    
   

In [ ]:
create_metadata_file()
start_tensorboard_p()

In [7]:
import tensorflow
import pandas as pd
import os
from sklearn.decomposition import PCA as sklearnPCA

def create_metadata_file(metadata, file_path):
    w = open(file_path, "w") 
    for elem in metadata:
        print(str(elem), file=w)
    w.close()

def start_tensorboard_p(X, n_components,directory,metadata_list): 
    PATH = os.getcwd()

    LOG_DIR = PATH + directory

    metadata = os.path.join(LOG_DIR, metadataFile)

    pca = sklearnPCA(n_components=n_components) #2-dimensional PCA
    pca_df = pd.DataFrame(pca.fit_transform(X))
    df_pca = pca_df.values

    tf_data = tf.Variable(df_pca)

    with tf.Session() as sess:
        saver = tf.train.Saver([tf_data])
        sess.run(tf.global_variables_initializer())
        saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
        config = projector.ProjectorConfig()

        embedding = config.embeddings.add()
        embedding.tensor_name = tf_data.name
        embedding.metadata_path = metadata

    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)